In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.signal import savgol_filter, wiener, medfilt

from keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D, LSTM
from keras.models import Sequential
from keras.utils import to_categorical, plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix as pcm

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

### Utils

In [ ]:
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)
        
def segment_signal(data, window_size = 64):
    segments = np.empty((0,window_size,6))
    labels = np.empty((0))
    for (start, end) in windows(data['timestamp'], window_size):
        acc_x = data['accX'][start:end]
        acc_y = data['accY'][start:end]
        acc_z = data['accZ'][start:end]
        gyr_x = data['gyrX'][start:end]
        gyr_y = data['gyrY'][start:end]
        gyr_z = data['gyrZ'][start:end]
        if(len(data['timestamp'][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z])])
            labels = np.append(labels,stats.mode(data['activityMode'][start:end])[0][0])
    return segments, labels

In [ ]:
def plot_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    ax[0].plot(history.epoch, history.history['accuracy'], label='accuracy')
    ax[0].plot(history.epoch, history.history['val_accuracy'], label='val accuracy')
    ax[1].plot(history.epoch, history.history['loss'], label='loss')
    ax[1].plot(history.epoch, history.history['val_loss'], label='val loss')
    ax[0].legend()
    ax[1].legend()
    plt.show()

In [ ]:
def plot_confusion_matrix(model, test_x, test_y):
    pred = model.predict(test_x)
    pred = np.argmax(pred, axis = 1) 
    y_true = np.argmax(test_y, axis = 1)

    cm = confusion_matrix(y_true, pred)
    fig, ax = pcm(conf_mat=cm, figsize=(10, 5))
    plt.show()

### Data preprocessing

In [ ]:
df_full = pd.read_csv('SensorData/sens.csv')
df_train = pd.read_csv('SensorData/train_sens.csv')

In [ ]:
df_full['timestamp'] = pd.to_datetime(df_full['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
df_full.head()



### Savitzky–Golay filter

In [ ]:
savgol = savgol_filter(x=df_full['accX'], window_length=31, polyorder=2)

In [ ]:
savgol_list = []
windows_length = range(3, 16, 2)
for i in windows_length:
    savgol_list.append(savgol_filter(x=df_full.loc[:100, 'accX'], window_length=i, polyorder=2))

In [ ]:
fig = plt.figure(figsize=(20, 6))
for i in savgol_list:
    ax = sns.lineplot(x=df_full.loc[:100, 'timestamp'], y=i)
ax.legend(windows_length)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df_full.loc[:1000, 'timestamp'], df_full.loc[:1000, 'accX'], label='original')
ax.plot(df_full.loc[:1000, 'timestamp'], savgol[:1001], 'r', label='filtered')
ax.set_title('Savitzky–Golay filter')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df_full['timestamp'], df_full['accX'], label='original')
ax.plot(df_full['timestamp'], savgol, 'r', label='filtered')
ax.set_title('Savitzky–Golay filter')
ax.legend()

### Wiener

In [ ]:
result_wiener = wiener(df_full['accX'], 35)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df_full.loc[:1000, 'timestamp'], df_full.loc[:1000, 'accX'], label='original')
ax.plot(df_full.loc[:1000, 'timestamp'], result_wiener[:1001], 'r', label='filtered')
ax.set_title('Wiener')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df_full['timestamp'], df_full['accX'], label='original')
ax.plot(df_full['timestamp'], result_wiener, 'r', label='filtered')
ax.set_title('Wiener')
ax.legend()

### Median filter

In [ ]:
result_medfilt = medfilt(df_full['accX'], kernel_size=45)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df_full.loc[:1000, 'timestamp'], df_full.loc[:1000, 'accX'], label='original')
ax.plot(df_full.loc[:1000, 'timestamp'], result_medfilt[:1001], 'r', label='filtered')
ax.set_title('Medfilt')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df_full['timestamp'], df_full['accX'], label='original')
ax.plot(df_full['timestamp'], result_medfilt, 'r', label='filtered')
ax.set_title('Medfilt')
ax.legend()

### Create model

### Ver. 1. CNN + Savitzky–Golay filter

#### window length = 55, polynom order = 3

In [ ]:
df_full.head()

In [ ]:
savgol_df = pd.DataFrame([])
for i, column in enumerate(df_full.columns[:-2]):
    savgol_df[column] = savgol_filter(x=df_full[column], window_length=55, polyorder=3)

In [ ]:
savgol_df[['activityMode', 'timestamp']] = df_full[['activityMode', 'timestamp']]

In [ ]:
savgol_df.head()

In [ ]:
fig, ax = plt.subplots(6, 1, figsize=(20, 14))
for i, column in enumerate(df.columns[:-2]):
    ax[i].plot(df_full['timestamp'], df_full[column])
    ax[i].plot(savgol_df['timestamp'], savgol_df[column])
    ax[i].set_title(column)

In [ ]:
savgol_segments, savgol_labels = segment_signal(savgol_df_train)
savgol_labels = np.asarray(pd.get_dummies(savgol_labels), dtype = np.int8)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(savgol_segments, savgol_labels, test_size=0.20)

In [ ]:
n_timestamps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(n_timestamps, n_features)))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y, epochs=90, batch_size=10, validation_split=0.1)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
def plot_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    ax[0].plot(history.epoch, history.history['accuracy'], label='accuracy')
    ax[0].plot(history.epoch, history.history['val_accuracy'], label='val accuracy')
    ax[1].plot(history.epoch, history.history['loss'], label='loss')
    ax[1].plot(history.epoch, history.history['val_loss'], label='val loss')
    ax[0].legend()
    ax[1].legend()

In [ ]:
plot_history(history)

In [ ]:
model.save('savgol-cnn-ver-1.h5')

In [ ]:
plot_confusion_matrix(model, test_x, test_y)

### Ver. 2. CNN + Wiener filter

In [ ]:
wiener_df = pd.DataFrame([])
for i, column in enumerate(df_full.columns[:-2]):
    wiener_df[column] = wiener(df_full[column], 55)

In [ ]:
wiener_df[['activityMode', 'timestamp']] = df[['activityMode', 'timestamp']]

In [ ]:
wiener_df.head()

In [ ]:
fig, ax = plt.subplots(6, 1, figsize=(20, 14))
for i, column in enumerate(df.columns[:-2]):
    ax[i].plot(df_full['timestamp'], df_full[column])
    ax[i].plot(wiener_df['timestamp'], wiener_df[column])
    ax[i].set_title(column)

In [ ]:
wiener_df_train = wiener_df[wiener_df['activityMode'] == 2].iloc[:5069]
wiener_df_train = wiener_df_train.append(wiener_df[wiener_df['activityMode'] == 0].iloc[:5069])
wiener_df_train = wiener_df_train.append(wiener_df[wiener_df['activityMode'] == 1].iloc[:5069])
wiener_df_train.reset_index(inplace=True)

In [ ]:
wiener_df_train['activityMode'].value_counts()

In [ ]:
wiener_segments, wiener_labels = segment_signal(wiener_df_train)
wiener_labels = np.asarray(pd.get_dummies(wiener_labels), dtype = np.int8)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(wiener_segments, wiener_labels, test_size=0.20)

In [ ]:
n_timestamps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(n_timestamps, n_features)))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y, epochs=100, batch_size=10, validation_split=0.1)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
plot_history(history)

In [ ]:
model.save('wiener-cnn-ver-1.h5')

In [ ]:
plot_confusion_matrix(model, test_x, test_y)

### Ver. 3. CNN + Median filter

In [ ]:
median_df = pd.DataFrame([])
for i, column in enumerate(df_full.columns[:-2]):
    median_df[column] = medfilt(df_full[column], kernel_size=55)

In [ ]:
median_df[['activityMode', 'timestamp']] = df_full[['activityMode', 'timestamp']]

In [ ]:
median_df.head()

In [ ]:
fig, ax = plt.subplots(6, 1, figsize=(20, 14))
for i, column in enumerate(df_full.columns[:-2]):
    ax[i].plot(df_full['timestamp'], df_full[column])
    ax[i].plot(median_df['timestamp'], median_df[column])
    ax[i].set_title(column)

In [ ]:
median_df_train = median_df[median_df['activityMode'] == 2].iloc[:5069]
median_df_train = median_df_train.append(median_df[median_df['activityMode'] == 0].iloc[:5069])
median_df_train = median_df_train.append(median_df[median_df['activityMode'] == 1].iloc[:5069])
median_df_train.reset_index(inplace=True)

In [ ]:
median_df_train['activityMode'].value_counts()

In [ ]:
median_segments, median_labels = segment_signal(median_df_train)
median_labels = np.asarray(pd.get_dummies(median_labels), dtype = np.int8)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(median_segments, median_labels, test_size=0.20)

In [ ]:
n_timestamps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(n_timestamps, n_features)))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y, epochs=100, batch_size=10, validation_split=0.1)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
plot_history(history)

In [ ]:
model.save('median-cnn-ver-1.h5')

In [ ]:
plot_confusion_matrix(model, test_x, test)